In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import math

def preprocess_data_train(df_x_og, df_y_og):
    cols_x = list(df_x_og.columns.values)
    stats_x = df_x_og.describe().loc[['mean', 'std']]
    stats_y = df_y_og.describe().loc[['mean']]
    ##standardize o-mean 1-variance
    def stand_x(x):
        return (x - stats_x[col][0]) / stats_x[col][1]

    def centre_y(y):
        return (y - stats_y[0])

    for col in cols_x:
        def stand_x(x):
            return (x - stats_x[col][0]) / stats_x[col][1]

        df_x_og[col] = df_x_og[col].apply(stand_x)
    df_y_og[0] = df_y_og[0].apply(centre_y)
    return df_x_og, df_y_og, stats_x, stats_y

def predic_test(test_x_test, train_mean_var_x, train_mean_y, w):
    test_x_new = pd.DataFrame()
    cols_x = list(test_x_test.columns.values)
    def stand_x(x):
        h = (x - train_mean_var_x[col][0]) / train_mean_var_x[col][1]
        return h 
    for col in cols_x:
        test_x_new[col] = test_x_test[col].apply(stand_x)
    w_trans = np.transpose(w)
    test_x_trans = np.transpose(test_x_new)
    # y_res=w^t*x^t
    y_res = np.dot(w_trans, test_x_trans)
    for i in range(13):
        y_res[0][i] = + train_mean_y[0]
    return y_res

def cross_validation():
    df_x = pd.read_csv('/Users/zhaoyuqing/desktop/698/a1/housing_X_train.csv', header=None)
    df_y = pd.read_csv('/Users/zhaoyuqing/desktop/698/a1/housing_y_train.csv', header=None)
    df_test_x = pd.read_csv('/Users/zhaoyuqing/desktop/698/a1/housing_X_test.csv', header=None)
    df_test_y = pd.read_csv('/Users/zhaoyuqing/desktop/698/a1/housing_y_test.csv', header=None)
    test_size = int(math.floor(len(df_x) * 0.1))  # Training set size: 90% of full data set.
    # iterate 10 times
    #lemada means lambda
    lemada = 0.1
    min_lemada = lemada
    # lemada_min_error to record the lemada corresponding to min average error
    min_error = 1000000
    while (lemada <= 50):
        error_sum = []  # calculate the mean of errors for one lemada
        for i in range(10):
            test_x = df_x.loc[len(df_x) - (i + 1) * test_size:len(df_x) - i * test_size]
            test_y = df_y.loc[len(df_y) - (i + 1) * test_size:len(df_y) - i * test_size]
            train_x = df_x.drop(df_x.index[len(df_x) - (i + 1) * test_size:len(df_x) - i * test_size])
            train_y = df_y.drop(df_y.index[len(df_y) - (i + 1) * test_size:len(df_y) - i * test_size])
            train_x_af, train_y_af, train_mean_var_x, train_mean_y = preprocess_data_train(train_x,train_y)  
            train_x_af_t = np.transpose(train_x_af.values)
            #(x^t*x+lemada*I)*w=x^t*y
            left = np.add(np.dot(train_x_af_t, train_x_af.values), np.dot(lemada, np.identity(13)))
            right = np.dot(train_x_af_t, train_y_af.values)
            w = np.linalg.solve(left, right)
            # predict test set
            predict_y_trains = predic_test(test_x, train_mean_var_x, train_mean_y, w)
            # calculate error per fold based on least square error
            predict_y = np.transpose(predict_y_trains)
            lstsq_fold = np.linalg.norm(np.subtract(predict_y, test_y))
            error_sum.append(lstsq_fold)
        lemada_error = np.asarray(error_sum).mean()
        if lemada_error < min_error:
            min_error = lemada_error
            min_lemada = lemada
        lemada = lemada + 0.01
    #At the last time, lemada has already+0.01    
    lemada = lemada - 0.01
    
#The best lambda is 49.99999999999862